In [1]:
from bs4 import BeautifulSoup
import requests

url = 'https://hospitals.webometrics.info/en/world'

# Send a GET request to the webpage
response = requests.get(url)
# Assuming `html_content` is the HTML source code you've posted.
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table by its class name.
table = soup.find('table', class_="sticky-enabled")

# Find all the rows in the table body, skipping the header row.
rows = table.find('tbody').find_all('tr')

# Create a list to hold all hospital data.
hospitals = []

# Loop through all the rows and get the text from each cell.
for row in rows:
    # Get all columns in the row.
    cols = row.find_all('td')
    website = cols[1].find('a')['href']
    # Extract text from each column using list comprehension.
    hospital_data = [col.text.strip() for col in cols]
    # Add a dictionary of the data to the hospitals list (optional).
    hospitals.append([
        hospital_data[1],
        
        website,
        
    ])

# Now you have a list of dictionaries with the data.
for hospital in hospitals:
    print(hospital)


['Cleveland Clinic', 'http://my.clevelandclinic.org/']
["St Jude Children's Research Hospital", 'http://www.stjude.org/']
['Johns Hopkins Medicine', 'http://www.hopkinsmedicine.org/']
['Mayo Clinic Scottsdale AZ', 'http://www.mayoclinic.org/']
['University of Maryland Medical Center', 'http://umm.edu/']
['M D Anderson Cancer Center', 'http://www.mdanderson.org/']
['Massachusetts General Hospital', 'http://www.massgeneral.org/']
['Assistance Publique Hôpitaux de Paris', 'http://www.aphp.fr/']
['Memorial Sloan Kettering Cancer Center', 'http://www.mskcc.org/']
['New York Presbyterian / Lower Manhattan Hospital', 'http://nyp.org/']
['Providence Health & Services', 'https://www2.providence.org/']
["Deer's Head Hospital Center", 'http://dhmh.maryland.gov/']
['Buddhist Tzu Chi General Hospital', 'http://www.tzuchi.com.tw/']
['Beth Israel Deaconess Medical Center Needham', 'http://www.bidmc.org/']
['Aurora Health Care', 'http://www.aurorahealthcare.org/']
['Taipei Veterans General Hospital', 

In [2]:
import pandas as pd

df = pd.DataFrame(columns = ['hospital_name', 'link'])
for i in hospitals:
    length = len(df)
    df.loc[length]= i
#df.to_csv(r'/Users/user/Desktop/ml-project/hospitals.csv', index = False)    

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import pandas as pd

def is_valid_url(url):
    """
    Checks if a given URL is valid and not an external link.
    """
    parsed_url = urlparse(url)
    return bool(parsed_url.netloc) and bool(parsed_url.scheme)

def get_all_website_links(url, domain_name):
    """
    Returns all URLs that are found on a website in which the domain name matches the domain name of the main URL.
    """
    urls = set()
    queue = [url]
    while queue:
        current_url = queue.pop(0)
        try:
            response = requests.get(current_url, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            for a_tag in soup.findAll("a"):
                href = a_tag.attrs.get("href")
                if href == "" or href is None:
                    # href empty tag
                    continue
                href = urljoin(current_url, href)
                parsed_href = urlparse(href)
                href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
                if not is_valid_url(href):
                    # not a valid URL
                    continue
                if domain_name not in href:
                    # external link
                    continue
                if href in urls:
                    continue
                urls.add(href)
                queue.append(href)
        except Exception as e:
            print(f"Error: {e}")
            continue
    return urls

def scrape_hospital_info(url):
    domain_name = urlparse(url).netloc
    all_links = get_all_website_links(url, domain_name)
    
    # Example of how you might process each page
    for link in all_links:
        response = requests.get(link)
        # You can add your scraping logic here, similar to the single-page example
    
# Example: Scraping for one hospital - you'll need to loop through your list for all hospitals
scrape_hospital_info("http://www.albertahealthservices.ca/")
'ok'

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import pandas as pd

# Initialize a session
session = requests.Session()
session.headers.update({'User-Agent': 'Custom User Agent'})

def scrape_site(url):
    """
    Scrape information from a hospital website starting with the given URL.
    This is a simplified example that primarily looks for contact information as a demonstration.
    """
    visited_urls = set()
    to_visit_urls = [url]

    hospital_info = {
        "Hospital Name": None,
        "Location (City)": None,
        "Location (State/Province)": None,
        "Location (Country)": None,
        "Contact Phone": None,
        "Website": url,
        "Email": None,
        "Number of Beds": None,
        "Specialties": None,
        "Awards/Accreditations": None,
        "Ranking": None,
        "Services": None,
        "Focus Areas": None,
        "Mission Statement": None,
        "History": None,
        "Doctor Names": None
    }

    while to_visit_urls:
        current_url = to_visit_urls.pop(0)
        if current_url in visited_urls:
            continue

        try:
            response = session.get(current_url)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Example: Looking for contact info
            # You would add specific logic here to extract the information of interest
            # This might include searching for specific tags, classes, or IDs that match what you're looking for

            visited_urls.add(current_url)

            # Find all links on page and add them to to_visit_urls if they match criteria
            for link in soup.find_all('a', href=True):
                href = link['href']
                full_url = urljoin(current_url, href)
                if urlparse(full_url).hostname == urlparse(url).hostname:
                    to_visit_urls.append(full_url)

        except requests.RequestException as e:
            print(f"Request failed for {current_url}: {e}")

    return hospital_info

# Replace 'your_hospital_website_here' with the actual hospital website URL
hospital_website = 'http://my.clevelandclinic.org/'
info = scrape_site(hospital_website)
print(info)
